In [1]:
!pip install -q -U keras-tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.8 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt


In [3]:
# Load the Fashion MNIST dataset
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

# Normalize pixel values to be between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [4]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    # Hyperparameter: Number of units in the Dense layer
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))

    model.add(keras.layers.Dense(10))

    # Hyperparameter: Learning rate for the optimizer
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    return model


In [5]:
tuner = kt.Hyperband(
    model_builder,
    objective='val_accuracy',  # Objective to optimize
    max_epochs=10,  # Maximum number of epochs
    factor=3,  # Reduces the number of trials to train at each step
    directory='my_dir',  # Directory to store logs
    project_name='fashion_mnist_tuning'  # Project name for the experiment
)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
""

''

In [7]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])


Trial 30 Complete [00h 01m 56s]
val_accuracy: 0.8806666731834412

Best val_accuracy So Far: 0.8899166584014893
Total elapsed time: 00h 21m 44s


In [8]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
The optimal number of units in the first dense layer is {best_hps.get('units')}
and the optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
""")



The optimal number of units in the first dense layer is 448 
and the optimal learning rate for the optimizer is 0.001.



In [9]:
# Build the model with the best hyperparameters
model = tuner.hypermodel.build(best_hps)

# Train the model with the optimal hyperparameters
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)


Epoch 1/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.7824 - loss: 0.6183 - val_accuracy: 0.8571 - val_loss: 0.3982
Epoch 2/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.8610 - loss: 0.3826 - val_accuracy: 0.8658 - val_loss: 0.3671
Epoch 3/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.8749 - loss: 0.3377 - val_accuracy: 0.8723 - val_loss: 0.3636
Epoch 4/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.8859 - loss: 0.3071 - val_accuracy: 0.8798 - val_loss: 0.3288
Epoch 5/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - accuracy: 0.8924 - loss: 0.2867 - val_accuracy: 0.8808 - val_loss: 0.3266
Epoch 6/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.8979 - loss: 0.2685 - val_accuracy: 0.8711 - val_loss: 0.3459
Epoch 7/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - accuracy: 0.9063 - loss: 0.2517 - val_accuracy: 0.8875 - val_loss: 0.3196
Epoch 8/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.9105 - loss: 0

In [1]:
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print(f'Best epoch: {best_epoch}')


NameError: name 'history' is not defined

In [2]:
# Train the model with the best hyperparameters (assuming you have best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

# After training, you can access 'history' and find the best epoch:
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print(f'Best epoch: {best_epoch}')


NameError: name 'model' is not defined

In [3]:
# Ensure you have the best hyperparameters from the tuner
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model using the best hyperparameters
model = tuner.hypermodel.build(best_hps)

# Train the model
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

# Access 'history' to find the best epoch
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print(f'Best epoch: {best_epoch}')


NameError: name 'tuner' is not defined

In [4]:
# Step 1: Install Keras Tuner
!pip install -q -U keras-tuner

# Step 2: Import necessary libraries
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

# Step 3: Load and prepare the dataset
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

# Step 4: Define the model builder function
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    # Hyperparameter: Number of units in the Dense layer
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))

    model.add(keras.layers.Dense(10))

    # Hyperparameter: Learning rate for the optimizer
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Step 5: Instantiate the tuner
tuner = kt.Hyperband(
    model_builder,
    objective='val_accuracy',  # What to optimize
    max_epochs=10,              # Maximum number of epochs
    factor=3,                   # Factor for adaptive resource allocation
    directory='my_dir',         # Directory to save results
    project_name='fashion_mnist_tuning'  # Project name for the experiment
)

# Step 6: Early stopping callback
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Step 7: Run the hyperparameter search
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Step 8: Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
The optimal number of units in the first dense layer is {best_hps.get('units')}
and the optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
""")

# Step 9: Build the model with the best hyperparameters
model = tuner.hypermodel.build(best_hps)

# Step 10: Train the model
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

# Step 11: Find the best epoch
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print(f'Best epoch: {best_epoch}')

# Step 12: Evaluate the model
eval_result = model.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)


Trial 30 Complete [00h 01m 16s]
val_accuracy: 0.11566666513681412

Best val_accuracy So Far: 0.718666672706604
Total elapsed time: 00h 19m 58s

The optimal number of units in the first dense layer is 320 
and the optimal learning rate for the optimizer is 0.0001.

Epoch 1/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.4377 - loss: 2.8479 - val_accuracy: 0.4098 - val_loss: 2.1946
Epoch 2/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - accuracy: 0.4126 - loss: 2.1970 - val_accuracy: 0.3236 - val_loss: 2.2820
Epoch 3/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.3066 - loss: 2.2798 - val_accuracy: 0.3016 - val_loss: 2.2743
Epoch 4/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.2985 - loss: 2.2718 - val_accuracy: 0.2083 - val_loss: 2.3043
Epoch 5/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.2244 - loss: 2.3027 - val_accuracy: 0.1982 - val_loss: 2.3031
Epoch 6/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.2367 - loss:

In [6]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first dense layer
    hp_units = hp.Int('units', min_value=64, max_value=256, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(64, activation='relu'))  # Additional layer for better feature extraction

    model.add(keras.layers.Dense(10, activation='softmax'))  # Change to softmax for classification

    # Tune the learning rate
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [7]:
import shutil
shutil.rmtree('my_dir', ignore_errors=True)


In [8]:
tuner = kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='my_dir',
    project_name='fashion_mnist_tuning'
)

tuner.search(img_train, label_train, epochs=50, validation_split=0.2)


Trial 29 Complete [00h 01m 27s]
val_accuracy: 0.8856666684150696

Best val_accuracy So Far: 0.890916645526886
Total elapsed time: 00h 17m 20s


In [9]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)


Epoch 1/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.7738 - loss: 0.6386 - val_accuracy: 0.8531 - val_loss: 0.3952
Epoch 2/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.8582 - loss: 0.3857 - val_accuracy: 0.8698 - val_loss: 0.3627
Epoch 3/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.8752 - loss: 0.3394 - val_accuracy: 0.8748 - val_loss: 0.3549
Epoch 4/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8849 - loss: 0.3095 - val_accuracy: 0.8748 - val_loss: 0.3494
Epoch 5/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.8883 - loss: 0.2949 - val_accuracy: 0.8791 - val_loss: 0.3295
Epoch 6/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.8984 - loss: 0.2755 - val_accuracy: 0.8839 - val_loss: 0.3398
Epoch 7/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.9012 - loss: 0.2647 - val_accuracy: 0.8864 - val_loss: 0.3248
Epoch 8/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.9065 - loss: 0.2500